In [1]:
import pandas as pd
from src.UsersClustering import UsersClustering
from src.AverageUser import AverageUser

In [9]:
%%capture output
sim_users = pd.read_csv("data/Prado_users_similarity_OnePolarity.csv", index_col=0)
users = pd.read_csv('data/Prado_users_emotions_OnePolarity.csv')

users['positive'] = users['positive'].apply(eval)
users['negative'] = users['negative'].apply(eval)
users['mixed'] = users['mixed'].apply(eval)

labels = UsersClustering().kMedoidsFromMatrix(sim_users)
users['cluster'] = labels

df = AverageUser().computeAverageUser(users)

In [10]:
df

,userId,age,gender,country,positive,negative,mixed,cluster
0,0,18-24,male,Spain,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57727cbaedc2cb3880e561f7, 577278d5edc2cb3880d...","[57727015edc2cb3880bceb08, 5772700eedc2cb3880b...",0
1,0,35-44,female,Spain,"[57726f59edc2cb3880ba76b3, 57727015edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f59edc2cb3880b...","[57727071edc2cb3880be3213, 57727015edc2cb3880b...",1
2,0,18-24,female,Other,"[57726f59edc2cb3880ba76b3, 5772752eedc2cb3880c...","[57727cbaedc2cb3880e561f7, 57726f76edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57727015edc2cb3880b...",2
3,0,45-54,male,Other,"[57727c95edc2cb3880e4d1d4, 57726f59edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f49edc2cb3880b...","[57727015edc2cb3880bceb08, 57726f59edc2cb3880b...",3
4,0,35-44,female,Spain,"[57726f59edc2cb3880ba76b3, 57727015edc2cb3880b...","[57726f4fedc2cb3880ba0b6d, 5772752eedc2cb3880c...","[57727cbaedc2cb3880e561f7, 57726f76edc2cb3880b...",4


In [2]:
import pandas as pd
import src.AverageUser as AU

In [3]:
users = pd.read_csv('data/Prado_users_clusters_OnePolarity.csv')
emotions = pd.read_csv('data/Prado_users_emotions_OnePolarity.csv')
users['positive'] = emotions['positive'].apply(eval)
users['negative'] = emotions['negative'].apply(eval)
users['mixed'] = emotions['mixed'].apply(eval)
AU.AverageUser().computeAverageUser(users)

,userId,age,gender,country,medoid,positive,negative,mixed,cluster
0,0,18-24,male,Spain,-1.0,"[57726f59edc2cb3880ba76b3, 57727c95edc2cb3880e...","[57727cbaedc2cb3880e561f7, 577278d5edc2cb3880d...","[57727015edc2cb3880bceb08, 5772700eedc2cb3880b...",0
1,0,35-44,female,Spain,-1.0,"[57726f59edc2cb3880ba76b3, 57727015edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f59edc2cb3880b...","[57727071edc2cb3880be3213, 57727015edc2cb3880b...",1
2,0,18-24,female,Other,-1.0,"[57726f59edc2cb3880ba76b3, 5772752eedc2cb3880c...","[57727cbaedc2cb3880e561f7, 57726f76edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57727015edc2cb3880b...",2
3,0,45-54,male,Other,-1.0,"[57727c95edc2cb3880e4d1d4, 57726f59edc2cb3880b...","[57727cbaedc2cb3880e561f7, 57726f49edc2cb3880b...","[57727015edc2cb3880bceb08, 57726f59edc2cb3880b...",3
4,0,35-44,female,Spain,-1.0,"[57726f59edc2cb3880ba76b3, 57727015edc2cb3880b...","[57726f4fedc2cb3880ba0b6d, 5772752eedc2cb3880c...","[57727cbaedc2cb3880e561f7, 57726f76edc2cb3880b...",4


In [1]:
%%capture output
!pip install ipywidgets
!pip install jupyterlab_widgets
# !pip install nodejs
# !pip install npm
!pip install widgetsnbextension
!jupyter nbextension enable --py widgetsnbextension

In [2]:
import ipywidgets as widgets

In [3]:
w = widgets.IntSlider()

In [4]:
display(w)

IntSlider(value=0)

In [5]:
display(w)

IntSlider(value=0)

In [6]:
a = widgets.FloatText()
b = widgets.FloatSlider()
display(a,b)

mylink = widgets.jslink((a, 'value'), (b, 'value'))

FloatText(value=0.0)

FloatSlider(value=0.0)

In [7]:
widgets.Dropdown(
    options=['1', '2', '3'],
    value='1',
    description='Number:',
    disabled=False,
)

Dropdown(description='Number:', options=('1', '2', '3'), value='1')

In [8]:
from os import listdir
from os.path import isfile, join

mypath = 'data'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

filesDropdown = widgets.Dropdown(
    options=files,
    value=files[0],
    description='Users:',
    disabled=False,
)
itemsDropdown = widgets.Dropdown(
    options=files,
    value=files[0],
    description='Items:',
    disabled=False,
)

display(filesDropdown)
display(itemsDropdown)

Dropdown(description='Users:', options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.csv', 'coun…

Dropdown(description='Items:', options=('artworkColors.json', 'artworkEmotions.json', 'Clean_users.csv', 'coun…

In [10]:
filesDropdown.value

'Prado_users_similarity.csv'